In [13]:
import time
timestamp = time.time()
print(timestamp)
sample_interval = 0.1
rounded_timestamp = round(timestamp / sample_interval) * sample_interval
print(rounded_timestamp)

1732295199.1446583
1732295199.1000001


In [14]:
next_time = time.time()
print(next_time)
next_time = next_time - (next_time % sample_interval)
print(next_time)

1732295247.892652
1732295247.8000002


In [18]:
import datetime
timestamp_str = datetime.datetime.fromtimestamp(timestamp).strftime('%H:%M:%S.%f')
print(timestamp_str)

17:06:39.144658


In [19]:
import os
import time
import threading
import subprocess
import pynvml
from datetime import datetime
from threading import Thread, Event
import argparse
import socket
import struct
import json

In [26]:
import threading
import time
import random

def thread_function(name, barrier, measurements,sample_interval=0.1):
    while not stop_event.is_set():
        # Wait until all threads have reached this point
        barrier.wait()
        # Take the measurement
        measurement = take_measurement(name)
        timestamp = time.time()
        next_time= timestamp + sample_interval
        rounded_timestamp = round((timestamp / sample_interval) * sample_interval, 1)
        measurements.append((rounded_timestamp, measurement))
        # Sleep to simulate work and control the measurement interval
        sleep_time = next_time - time.time()
        if sleep_time > 0:
            time.sleep(sleep_time)
        else:
            pass

def take_measurement(thread_name):
    # Replace this with actual measurement code
    return f"Measurement from {thread_name} {round(random.random(),1)}"

# Parameters
measurement_interval = 0.1  # Time between measurements in seconds
total_runtime = 1  # Total time to run the measurement in seconds

# Synchronization primitives
num_threads = 2
barrier = threading.Barrier(num_threads)
stop_event = threading.Event()

# Data storage
measurements_a = []
measurements_b = []

# Create and start threads
thread_a = threading.Thread(target=thread_function, args=("Thread A", barrier, measurements_a))
thread_b = threading.Thread(target=thread_function, args=("Thread B", barrier, measurements_b))
thread_a.start()
thread_b.start()

# Let the threads run for the specified total runtime
time.sleep(total_runtime)
stop_event.set()

# Wait for threads to finish
thread_a.join()
thread_b.join()

# Print measurements
print("Measurements from Thread A:")
for timestamp, measurement in measurements_a:
    print(f"Time: {timestamp}, Measurement: {measurement}")

print("\nMeasurements from Thread B:")
for timestamp, measurement in measurements_b:
    print(f"Time: {timestamp}, Measurement: {measurement}")


Measurements from Thread A:
Time: 1732302022.6, Measurement: Measurement from Thread A 0.7
Time: 1732302022.7, Measurement: Measurement from Thread A 0.9
Time: 1732302022.8, Measurement: Measurement from Thread A 0.8
Time: 1732302022.9, Measurement: Measurement from Thread A 0.7
Time: 1732302023.0, Measurement: Measurement from Thread A 0.6
Time: 1732302023.1, Measurement: Measurement from Thread A 0.5
Time: 1732302023.2, Measurement: Measurement from Thread A 1.0
Time: 1732302023.3, Measurement: Measurement from Thread A 0.8
Time: 1732302023.4, Measurement: Measurement from Thread A 0.1
Time: 1732302023.5, Measurement: Measurement from Thread A 0.9

Measurements from Thread B:
Time: 1732302022.6, Measurement: Measurement from Thread B 0.5
Time: 1732302022.7, Measurement: Measurement from Thread B 0.5
Time: 1732302022.8, Measurement: Measurement from Thread B 0.9
Time: 1732302022.9, Measurement: Measurement from Thread B 0.6
Time: 1732302023.0, Measurement: Measurement from Thread B 0.

In [2]:
import threading
import time
import random
from queue import Queue, Empty

def thread_function(name, barrier, measurements_queue, sample_interval=0.1):
    while not stop_event.is_set():
        # Wait until all threads have reached this point
        barrier.wait()
        timestamp = time.time()
        # Round the timestamp to align to sample_interval
        rounded_timestamp = round((timestamp / sample_interval) * sample_interval, 1)
        # Take the measurement
        measurement = take_measurement(name)
        measurements_queue.put((rounded_timestamp, measurement))
        # Sleep to control the measurement interval
        next_time = timestamp + sample_interval
        sleep_time = next_time - time.time()
        if sleep_time > 0:
            time.sleep(sleep_time)
        else:
            pass

def take_measurement(thread_name):
    # Replace this with actual measurement code
    return f"Measurement from {thread_name} {round(random.random(),1)}"

def accumulator_function(measurements_a, measurements_b, sample_interval):
    data_a = {}
    data_b = {}
    processed_timestamps = set()
    last_measurement_a = None
    last_measurement_b = None

    while not stop_event.is_set() or not measurements_a.empty() or not measurements_b.empty():
        # Try to get new measurements from the queues
        try:
            while True:
                timestamp_a, measurement_a = measurements_a.get_nowait()
                data_a[timestamp_a] = measurement_a
        except Empty:
            pass

        try:
            while True:
                timestamp_b, measurement_b = measurements_b.get_nowait()
                data_b[timestamp_b] = measurement_b
        except Empty:
            pass

        # Get all timestamps that have measurements from at least one thread
        all_timestamps = set(data_a.keys()) | set(data_b.keys())
        pending_timestamps = sorted(all_timestamps - processed_timestamps)

        for timestamp in pending_timestamps:
            measurement_a = data_a.get(timestamp)
            measurement_b = data_b.get(timestamp)

            # If we have measurements from both threads, process them
            if measurement_a is not None and measurement_b is not None:
                print(f"Time: {timestamp}, Measurement A: {measurement_a}, Measurement B: {measurement_b}")
                last_measurement_a = measurement_a
                last_measurement_b = measurement_b
                processed_timestamps.add(timestamp)
                del data_a[timestamp]
                del data_b[timestamp]
            else:
                # If we don't have both measurements, check if we should fill in missing data
                time_since_timestamp = time.time() - timestamp
                if time_since_timestamp > sample_interval * 2:
                    # Fill in missing measurements using last known values
                    if measurement_a is None:
                        measurement_a = last_measurement_a
                    else:
                        last_measurement_a = measurement_a
                    if measurement_b is None:
                        measurement_b = last_measurement_b
                    else:
                        last_measurement_b = measurement_b

                    print(f"Time: {timestamp}, Measurement A: {measurement_a}, Measurement B: {measurement_b} (Filled)")
                    processed_timestamps.add(timestamp)
                    if timestamp in data_a:
                        del data_a[timestamp]
                    if timestamp in data_b:
                        del data_b[timestamp]

        time.sleep(0.01)

# Parameters
measurement_interval = 0.1  # Time between measurements in seconds
total_runtime = 1  # Total time to run the measurement in seconds

# Synchronization primitives
num_threads = 2
barrier = threading.Barrier(num_threads)
stop_event = threading.Event()

# Data storage
measurements_a = Queue()
measurements_b = Queue()

# Create and start threads
thread_a = threading.Thread(target=thread_function, args=("Thread A", barrier, measurements_a, measurement_interval))
thread_b = threading.Thread(target=thread_function, args=("Thread B", barrier, measurements_b, measurement_interval))
accumulator_thread = threading.Thread(target=accumulator_function, args=(measurements_a, measurements_b, measurement_interval))

thread_a.start()
thread_b.start()
accumulator_thread.start()

# Let the threads run for the specified total runtime
time.sleep(total_runtime)
stop_event.set()

# Wait for threads to finish
thread_a.join()
thread_b.join()
accumulator_thread.join()


Time: 1732303697.5, Measurement A: Measurement from Thread A 1.0, Measurement B: Measurement from Thread B 0.1
Time: 1732303697.6, Measurement A: Measurement from Thread A 0.3, Measurement B: Measurement from Thread B 0.8
Time: 1732303697.7, Measurement A: Measurement from Thread A 0.9, Measurement B: Measurement from Thread B 0.2
Time: 1732303697.8, Measurement A: Measurement from Thread A 0.1, Measurement B: Measurement from Thread B 0.1
Time: 1732303697.9, Measurement A: Measurement from Thread A 1.0, Measurement B: Measurement from Thread B 0.7
Time: 1732303698.0, Measurement A: Measurement from Thread A 0.4, Measurement B: Measurement from Thread B 0.3
Time: 1732303698.1, Measurement A: Measurement from Thread A 0.4, Measurement B: Measurement from Thread B 0.6
Time: 1732303698.2, Measurement A: Measurement from Thread A 0.7, Measurement B: Measurement from Thread B 0.0
Time: 1732303698.3, Measurement A: Measurement from Thread A 0.6, Measurement B: Measurement from Thread B 0.0
T

In [25]:
batch =[(1732332507.4, {'cpu_memory': {'memory_energy': 1.37, 'cpu_energy': 8.92}, 'gpu': {'gpu_energy': 1.68}}), (1732332507.3, {'cpu_memory': {'memory_energy': 1.37, 'cpu_energy': 8.99}, 'gpu': {'gpu_energy': 1.82}}), (1732332507.6, {'cpu_memory': {'memory_energy': 1.37, 'cpu_energy': 8.92}, 'gpu': {'gpu_energy': 1.68}}), (1732332507.7, {'cpu_memory': {'memory_energy': 1.36, 'cpu_energy': 8.97}, 'gpu': {'gpu_energy': 1.68}}), (1732332507.8, {'cpu_memory': {'memory_energy': 1.36, 'cpu_energy': 8.93}, 'gpu': {'gpu_energy': 1.87}}), (1732332508.0, {'cpu_memory': {'memory_energy': 1.47, 'cpu_energy': 9.59}, 'gpu': {'gpu_energy': 1.87}}), (1732332508.1, {'cpu_memory': {'memory_energy': 1.37, 'cpu_energy': 8.94}, 'gpu': {'gpu_energy': 1.87}}), (1732332508.2, {'cpu_memory': {'memory_energy': 1.36, 'cpu_energy': 8.93}, 'gpu': {'gpu_energy': 1.87}}), (1732332508.4, {'cpu_memory': {'memory_energy': 1.38, 'cpu_energy': 9.0}, 'gpu': {'gpu_energy': 1.87}}), (1732332508.5, {'cpu_memory': {'memory_energy': 1.37, 'cpu_energy': 8.93}, 'gpu': {'gpu_energy': 1.87}}), (1732332508.7, {'cpu_memory': {'memory_energy': 1.37, 'cpu_energy': 8.97}, 'gpu': {'gpu_energy': 1.87}}), (1732332508.8, {'cpu_memory': {'memory_energy': 1.37, 'cpu_energy': 8.94}, 'gpu': {'gpu_energy': 1.87}}), (1732332508.9, {'cpu_memory': {'memory_energy': 1.36, 'cpu_energy': 8.92}, 'gpu': {'gpu_energy': 1.74}}), (1732332509.1, {'cpu_memory': {'memory_energy': 1.35, 'cpu_energy': 8.93}, 'gpu': {'gpu_energy': 1.74}}), (1732332509.2, {'cpu_memory': {'memory_energy': 1.35, 'cpu_energy': 8.97}, 'gpu': {'gpu_energy': 1.74}}), (1732332509.3, {'cpu_memory': {'memory_energy': 1.36, 'cpu_energy': 8.98}, 'gpu': {'gpu_energy': 1.78}}), (1732332509.5, {'cpu_memory': {'memory_energy': 1.34, 'cpu_energy': 8.93}, 'gpu': {'gpu_energy': 1.78}}), (1732332509.6, {'cpu_memory': {'memory_energy': 1.36, 'cpu_energy': 8.96}, 'gpu': {'gpu_energy': 1.78}}), (1732332509.8, {'cpu_memory': {'memory_energy': 1.38, 'cpu_energy': 8.97}, 'gpu': {'gpu_energy': 1.78}}), (1732332509.9, {'cpu_memory': {'memory_energy': 1.37, 'cpu_energy': 8.93}, 'gpu': {'gpu_energy': 1.78}})]

In [26]:
for i in batch:
    print(i)

(1732332507.4, {'cpu_memory': {'memory_energy': 1.37, 'cpu_energy': 8.92}, 'gpu': {'gpu_energy': 1.68}})
(1732332507.3, {'cpu_memory': {'memory_energy': 1.37, 'cpu_energy': 8.99}, 'gpu': {'gpu_energy': 1.82}})
(1732332507.6, {'cpu_memory': {'memory_energy': 1.37, 'cpu_energy': 8.92}, 'gpu': {'gpu_energy': 1.68}})
(1732332507.7, {'cpu_memory': {'memory_energy': 1.36, 'cpu_energy': 8.97}, 'gpu': {'gpu_energy': 1.68}})
(1732332507.8, {'cpu_memory': {'memory_energy': 1.36, 'cpu_energy': 8.93}, 'gpu': {'gpu_energy': 1.87}})
(1732332508.0, {'cpu_memory': {'memory_energy': 1.47, 'cpu_energy': 9.59}, 'gpu': {'gpu_energy': 1.87}})
(1732332508.1, {'cpu_memory': {'memory_energy': 1.37, 'cpu_energy': 8.94}, 'gpu': {'gpu_energy': 1.87}})
(1732332508.2, {'cpu_memory': {'memory_energy': 1.36, 'cpu_energy': 8.93}, 'gpu': {'gpu_energy': 1.87}})
(1732332508.4, {'cpu_memory': {'memory_energy': 1.38, 'cpu_energy': 9.0}, 'gpu': {'gpu_energy': 1.87}})
(1732332508.5, {'cpu_memory': {'memory_energy': 1.37, 'c

In [27]:
batch.sort(key=lambda x: x[0])
for i in batch:
    print(i)
timestamps = [entry[0] for entry in batch]
missed_timestamps=[]
for i in range(len(timestamps) - 1):
    if timestamps[i + 1] * 10 - timestamps[i] * 10 == 2:
        missed_timestamps.append((timestamps[i] * 10 + 1)/10)
        # print((timestamps[i] * 10 + 1)/10)
print(missed_timestamps)

(1732332507.3, {'cpu_memory': {'memory_energy': 1.37, 'cpu_energy': 8.99}, 'gpu': {'gpu_energy': 1.82}})
(1732332507.4, {'cpu_memory': {'memory_energy': 1.37, 'cpu_energy': 8.92}, 'gpu': {'gpu_energy': 1.68}})
(1732332507.6, {'cpu_memory': {'memory_energy': 1.37, 'cpu_energy': 8.92}, 'gpu': {'gpu_energy': 1.68}})
(1732332507.7, {'cpu_memory': {'memory_energy': 1.36, 'cpu_energy': 8.97}, 'gpu': {'gpu_energy': 1.68}})
(1732332507.8, {'cpu_memory': {'memory_energy': 1.36, 'cpu_energy': 8.93}, 'gpu': {'gpu_energy': 1.87}})
(1732332508.0, {'cpu_memory': {'memory_energy': 1.47, 'cpu_energy': 9.59}, 'gpu': {'gpu_energy': 1.87}})
(1732332508.1, {'cpu_memory': {'memory_energy': 1.37, 'cpu_energy': 8.94}, 'gpu': {'gpu_energy': 1.87}})
(1732332508.2, {'cpu_memory': {'memory_energy': 1.36, 'cpu_energy': 8.93}, 'gpu': {'gpu_energy': 1.87}})
(1732332508.4, {'cpu_memory': {'memory_energy': 1.38, 'cpu_energy': 9.0}, 'gpu': {'gpu_energy': 1.87}})
(1732332508.5, {'cpu_memory': {'memory_energy': 1.37, 'c

In [28]:
def fill_missing_timestamps(list_of_tuples, missing_timestamps):
    """
    Fills the missing timestamps in the list of tuples with measurements copied
    from the nearest timestamp.
    """
    # Sort the original list by timestamps
    list_of_tuples.sort(key=lambda x: x[0])
    
    # Combine original timestamps and measurements into a dictionary for easy access
    timestamp_to_measurement = {timestamp: measurement for timestamp, measurement in list_of_tuples}

    # Process each missing timestamp
    for missing in sorted(missing_timestamps):
        # Find the closest timestamp in the existing list
        closest_timestamp = min(timestamp_to_measurement.keys(), key=lambda t: abs(t - missing))
        # Copy the measurement of the closest timestamp to the missing timestamp
        timestamp_to_measurement[missing] = timestamp_to_measurement[closest_timestamp]

    # Create the new list of tuples, sorted by timestamps
    filled_list = sorted(timestamp_to_measurement.items(), key=lambda x: x[0])
    return filled_list

f_batch=fill_missing_timestamps(batch,missed_timestamps)
for i in f_batch:
    print(i)

(1732332507.3, {'cpu_memory': {'memory_energy': 1.37, 'cpu_energy': 8.99}, 'gpu': {'gpu_energy': 1.82}})
(1732332507.4, {'cpu_memory': {'memory_energy': 1.37, 'cpu_energy': 8.92}, 'gpu': {'gpu_energy': 1.68}})
(1732332507.5, {'cpu_memory': {'memory_energy': 1.37, 'cpu_energy': 8.92}, 'gpu': {'gpu_energy': 1.68}})
(1732332507.6, {'cpu_memory': {'memory_energy': 1.37, 'cpu_energy': 8.92}, 'gpu': {'gpu_energy': 1.68}})
(1732332507.7, {'cpu_memory': {'memory_energy': 1.36, 'cpu_energy': 8.97}, 'gpu': {'gpu_energy': 1.68}})
(1732332507.8, {'cpu_memory': {'memory_energy': 1.36, 'cpu_energy': 8.93}, 'gpu': {'gpu_energy': 1.87}})
(1732332507.9, {'cpu_memory': {'memory_energy': 1.47, 'cpu_energy': 9.59}, 'gpu': {'gpu_energy': 1.87}})
(1732332508.0, {'cpu_memory': {'memory_energy': 1.47, 'cpu_energy': 9.59}, 'gpu': {'gpu_energy': 1.87}})
(1732332508.1, {'cpu_memory': {'memory_energy': 1.37, 'cpu_energy': 8.94}, 'gpu': {'gpu_energy': 1.87}})
(1732332508.2, {'cpu_memory': {'memory_energy': 1.36, '

In [31]:
consolidated_data = [
            {
                'node_id': 'compute2',
                'timestamp': timestamp,
                'measurement': measurement
            }
            for timestamp, measurement in f_batch
        ]
print(consolidated_data, len(consolidated_data))

[{'node_id': 'compute2', 'timestamp': 1732332507.3, 'measurement': {'cpu_memory': {'memory_energy': 1.37, 'cpu_energy': 8.99}, 'gpu': {'gpu_energy': 1.82}}}, {'node_id': 'compute2', 'timestamp': 1732332507.4, 'measurement': {'cpu_memory': {'memory_energy': 1.37, 'cpu_energy': 8.92}, 'gpu': {'gpu_energy': 1.68}}}, {'node_id': 'compute2', 'timestamp': 1732332507.5, 'measurement': {'cpu_memory': {'memory_energy': 1.37, 'cpu_energy': 8.92}, 'gpu': {'gpu_energy': 1.68}}}, {'node_id': 'compute2', 'timestamp': 1732332507.6, 'measurement': {'cpu_memory': {'memory_energy': 1.37, 'cpu_energy': 8.92}, 'gpu': {'gpu_energy': 1.68}}}, {'node_id': 'compute2', 'timestamp': 1732332507.7, 'measurement': {'cpu_memory': {'memory_energy': 1.36, 'cpu_energy': 8.97}, 'gpu': {'gpu_energy': 1.68}}}, {'node_id': 'compute2', 'timestamp': 1732332507.8, 'measurement': {'cpu_memory': {'memory_energy': 1.36, 'cpu_energy': 8.93}, 'gpu': {'gpu_energy': 1.87}}}, {'node_id': 'compute2', 'timestamp': 1732332507.9, 'measu